In [1]:
# @Time     : Jan. 02, 2019 22:17
# @Author   : Veritas YIN
# @FileName : main.py
# @Version  : 1.0
# @Project  : Orion
# @IDE      : PyCharm
# @Github   : https://github.com/VeritasYin/Project_Orion

import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from os.path import join as pjoin

import tensorflow as tf

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
tf.Session(config=config)

from utils.math_graph import *
from data_loader.data_utils import *
from models.trainer import model_train
from models.tester import model_test

import argparse

parser = argparse.ArgumentParser()
parser.add_argument('--n_route', type=int, default=228)
parser.add_argument('--n_his', type=int, default=12)
parser.add_argument('--n_pred', type=int, default=9)
parser.add_argument('--batch_size', type=int, default=50)
parser.add_argument('--epoch', type=int, default=50)
parser.add_argument('--save', type=int, default=10)
parser.add_argument('--ks', type=int, default=3)
parser.add_argument('--kt', type=int, default=3)
parser.add_argument('--lr', type=float, default=1e-3)
parser.add_argument('--opt', type=str, default='RMSProp')
parser.add_argument('--graph', type=str, default='default')
parser.add_argument('--inf_mode', type=str, default='merge')
parser.add_argument("-f", type=str, default="")

args = parser.parse_args()
print(f'Training configs: {args}')

n, n_his, n_pred = args.n_route, args.n_his, args.n_pred
Ks, Kt = args.ks, args.kt
# blocks: settings of channel size in st_conv_blocks / bottleneck design
blocks = [[1, 32, 64], [64, 32, 128]]

# Load wighted adjacency matrix W
if args.graph == 'default':
    #W = weight_matrix(pjoin('./dataset', f'PeMSD7_W_{n}.csv'))
    W = weight_matrix(pjoin('./dataset', f'W_{n}.csv'))
else:
    # load customized graph weight matrix
    W = weight_matrix(pjoin('./dataset', args.graph))

# Calculate graph kernel
L = scaled_laplacian(W)
# Alternative approximation method: 1st approx - first_approx(W, n).
Lk = cheb_poly_approx(L, Ks, n)
tf.add_to_collection(name='graph_kernel', value=tf.cast(tf.constant(Lk), tf.float32))

# Data Preprocessing
#data_file = f'PeMSD7_V_{n}.csv'
data_file = f'V_{n}.csv'
n_train, n_val, n_test = 34, 5, 5
PeMS = data_gen(pjoin('./dataset', data_file), (n_train, n_val, n_test), n, n_his + n_pred)
print(f'>> Loading dataset with Mean: {PeMS.mean:.2f}, STD: {PeMS.std:.2f}')



Training configs: Namespace(batch_size=50, epoch=50, f='/home/rodr/.local/share/jupyter/runtime/kernel-06739724-6665-4127-a30c-cc814db61378.json', graph='default', inf_mode='merge', ks=3, kt=3, lr=0.001, n_his=12, n_pred=9, n_route=228, opt='RMSProp', save=10)



2022-02-09 23:22:38.827293: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2022-02-09 23:22:38.851261: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 3693290000 Hz
2022-02-09 23:22:38.853379: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x564686d4edf0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-02-09 23:22:38.853400: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


>> Loading dataset with Mean: 58.50, STD: 13.73


In [2]:
model_train(PeMS, blocks, args)







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor



Epoch  0, Step   0: [4467.932, 140.087]
Epoch  0, Step  50: [453.114, 171.857]
Epoch  0, Step 100: [704.625, 177.481]
Epoch  0, Step 150: [231.732, 178.909]
Epoch  0 Training Time 41.770s
Time Step 3: MAPE  6.837%,  6.614%; MAE  2.944, 2.928; RMSE  4.945,  4.911.
Time Step 6: MAPE  9.778%,  9.512%; MAE  4.230, 4.220; RMSE  7.324,  7.303.
Time Step 9: MAPE 12.074%, 11.727%; MAE  5.227, 5.211; RMSE  9.047,  9.059.
Epoch  0 Inference Time 37.072s
Epoch  1, Step   0: [184.644, 173.594]
Epoch  1, Step  50: [389.291, 173.568]
Epoch  1, Step 100: [219.402, 168.743]
Epoch  1, Step 150: [197.356, 192.804]
Epoch  1 Training Time 41.779s
Time Step 3: MAPE  6.837%,  6.701%; MAE  2.914, 2.893; RMSE  4.807,  4.811.
Time Step 6: MAPE  9.778%,  9.794%; MAE  4.1

Epoch 11, Step   0: [115.876, 150.779]
Epoch 11, Step  50: [138.987, 183.360]
Epoch 11, Step 100: [152.073, 179.156]
Epoch 11, Step 150: [130.883, 160.829]
Epoch 11 Training Time 42.016s
Time Step 3: MAPE  5.612%,  5.512%; MAE  2.322, 2.342; RMSE  4.172,  4.207.
Time Step 6: MAPE  8.347%,  8.104%; MAE  3.234, 3.270; RMSE  6.023,  6.056.
Time Step 9: MAPE 10.323%, 10.381%; MAE  3.989, 4.020; RMSE  7.409,  7.431.
Epoch 11 Inference Time 18.813s
Epoch 12, Step   0: [156.782, 202.959]
Epoch 12, Step  50: [137.678, 177.500]
Epoch 12, Step 100: [130.251, 170.112]
Epoch 12, Step 150: [126.779, 161.611]
Epoch 12 Training Time 42.046s
Time Step 3: MAPE  5.484%,  5.455%; MAE  2.322, 2.388; RMSE  4.172,  4.255.
Time Step 6: MAPE  7.887%,  7.847%; MAE  3.234, 3.378; RMSE  6.023,  6.216.
Time Step 9: MAPE  9.754%,  9.634%; MAE  3.989, 4.129; RMSE  7.409,  7.608.
Epoch 12 Inference Time 37.035s
Epoch 13, Step   0: [131.856, 175.497]
Epoch 13, Step  50: [130.282, 158.332]
Epoch 13, Step 100: [137.347

Epoch 29, Step 150: [119.764, 165.858]
Epoch 29 Training Time 45.016s
Time Step 3: MAPE  5.264%,  5.276%; MAE  2.211, 2.236; RMSE  4.030,  4.064.
Time Step 6: MAPE  7.451%,  7.529%; MAE  3.000, 3.055; RMSE  5.715,  5.773.
Time Step 9: MAPE  9.057%,  9.213%; MAE  3.579, 3.658; RMSE  6.859,  6.929.
Epoch 29 Inference Time 20.187s
<< Saving model to ./output/models/STGCN-5490 ...
Epoch 30, Step   0: [127.100, 178.351]
Epoch 30, Step  50: [112.298, 159.568]
Epoch 30, Step 100: [141.791, 205.418]
Epoch 30, Step 150: [121.951, 177.162]
Epoch 30 Training Time 44.849s
Time Step 3: MAPE  5.264%,  5.276%; MAE  2.211, 2.236; RMSE  4.030,  4.064.
Time Step 6: MAPE  7.451%,  7.529%; MAE  3.000, 3.055; RMSE  5.715,  5.773.
Time Step 9: MAPE  9.057%,  9.213%; MAE  3.579, 3.658; RMSE  6.859,  6.929.
Epoch 30 Inference Time 20.025s
Epoch 31, Step   0: [123.852, 191.145]
Epoch 31, Step  50: [110.728, 163.274]
Epoch 31, Step 100: [128.101, 172.780]
Epoch 31, Step 150: [116.022, 161.138]
Epoch 31 Training

Time Step 3: MAPE  5.231%,  5.238%; MAE  2.211, 2.249; RMSE  4.030,  4.069.
Time Step 6: MAPE  7.380%,  7.355%; MAE  2.992, 3.080; RMSE  5.692,  5.766.
Time Step 9: MAPE  8.963%,  8.844%; MAE  3.536, 3.688; RMSE  6.773,  6.911.
Epoch 47 Inference Time 37.867s
Epoch 48, Step   0: [125.282, 178.932]
Epoch 48, Step  50: [116.108, 173.294]
Epoch 48, Step 100: [102.061, 150.323]
Epoch 48, Step 150: [128.369, 180.062]
Epoch 48 Training Time 42.552s
Time Step 3: MAPE  5.231%,  5.238%; MAE  2.211, 2.249; RMSE  4.030,  4.069.
Time Step 6: MAPE  7.380%,  7.355%; MAE  2.992, 3.080; RMSE  5.692,  5.766.
Time Step 9: MAPE  8.963%,  8.844%; MAE  3.536, 3.688; RMSE  6.773,  6.911.
Epoch 48 Inference Time 18.783s
Epoch 49, Step   0: [121.794, 181.583]
Epoch 49, Step  50: [113.840, 170.669]
Epoch 49, Step 100: [130.730, 191.984]
Epoch 49, Step 150: [126.201, 177.247]
Epoch 49 Training Time 42.525s
Time Step 3: MAPE  5.221%,  5.217%; MAE  2.211, 2.241; RMSE  4.030,  4.078.
Time Step 6: MAPE  7.373%,  7.

In [4]:
model_test(PeMS, PeMS.get_len('test'), n_his, n_pred, args.inf_mode)


INFO:tensorflow:Restoring parameters from ./output/models/STGCN-9150
>> Loading saved model from ./output/models/STGCN-9150 ...
Time Step 3: MAPE  5.217%; MAE  2.241; RMSE  4.078.
Time Step 6: MAPE  7.333%; MAE  3.063; RMSE  5.780.
Time Step 9: MAPE  8.848%; MAE  3.659; RMSE  6.930.
Model Test Time 24.171s
Testing model finished!
